In [ ]:
import sys
lib_path = [r'C:\Users\ikahbasi\OneDrive\Applications\GitHub\SeisRoutine',
            r'C:\Users\ikahb\OneDrive\Applications\GitHub\SeisRoutine']
for path in lib_path:
    sys.path.append(path)
##########################################################################
import SeisRoutine.catalog as src
import SeisRoutine.waveform as srw
import SeisRoutine.config as srconf
import SeisRoutine.statistics as srs

In [ ]:
import seisbench.data as sbd
import seisbench.generate as sbg
import numpy as np
import os
from scipy import signal
from tqdm import tqdm

In [ ]:
init_cfg = srconf.load_config('0-init-cfg.yml')
cfg_path = os.path.join(init_cfg.target_config_filepath,
                        init_cfg.target_config_filename)
cfg = srconf.load_config(cfg_path)

In [ ]:
dataset = sbd.WaveformDataset(
    path=cfg.dataset.path,
    sampling_rate=cfg.training.dataset.sampling_rate,
    component_order=cfg.training.dataset.component_order,
          )

In [ ]:
class Tapering:
    def __init__(self, alpha=0.3, key='X'):
        self.alpha = alpha  # Tapering Coefficient
        if isinstance(key, str):
            self.key = (key, key)
        else:
            self.key = key

    def __call__(self, state_dict):
        x, metadata = state_dict[self.key[0]]
        taper = signal.windows.tukey(x.shape[-1], self.alpha)
        x = x * taper
        state_dict[self.key[1]] = (x, metadata)

In [ ]:
sps = 100
augmentations = [
    # Tapering(),
    # sbg.Filter(N=4,
    #            Wn=[1],
    #            btype='highpass',
    #            forward_backward=True,
    #            ),
    sbg.Normalize(
        demean_axis=-1,
        amp_norm_axis=-1,
        amp_norm_type="peak"),
    sbg.ChangeDtype(np.float32),
]
generator = sbg.GenericGenerator(dataset)
generator.add_augmentations(augmentations)

In [ ]:
metadata = dataset.metadata.copy()
sps = dataset.sampling_rate
for ii in tqdm(metadata.index):
    data_3c, _ = dataset.get_sample(ii)
    for data, channel in zip(data_3c, dataset.component_order):
        freq, ampl = srw.waveform.fft(array=data, delta=1/sps)
        fft_low = ampl[freq<1]
        fft_mid = ampl[(1<=freq) & (freq<20)]
        fft_hig = ampl[20<=freq]
        metadata.at[ii, f'trace_{channel}_max_L-band_fft'] = fft_low.max().round(3)
        metadata.at[ii, f'trace_{channel}_max_M-band_fft'] = fft_mid.max().round(3)
        metadata.at[ii, f'trace_{channel}_max_H-band_fft'] = fft_hig.max().round(3)
    # break

In [ ]:
path = os.path.join(cfg.dataset.path, 'metadata-with-frequency.pkl')
metadata.to_pickle(path)